# Jupyter pglite anywidget demo

Demo of using `jupyterlite_anywidget_pglite` widget and magics.

Works in:

- Jupyter UIs (JupyterLab, Jupyter notebook)
- VS Code (via Jupyter notebook support)

In [23]:
# Package installation required in JupyterLite pyodide kernel
# %pip install jupyter_anywidget_pglite anywidget==0.9.13

## Headless Demo

In the headless demo, no HTML UI elements are added to the DOM, but the `anywidget` machinery *is* added via a call to `IPython.display.display`.

In [1]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_headless

In [2]:
pg_headless = pglite_headless()

postgresWidget(headless=True, response={'status': 'initialising'})

In [3]:
pg_headless.query("SELECT 'hello';")

{'status': 'completed',
 'response': {'rows': [{'?column?': 'hello'}],
  'fields': [{'name': '?column?', 'dataTypeID': 25}],
  'affectedRows': 0},
 'response_type': 'single'}

In [4]:
pg_headless.response

{'status': 'completed',
 'response': {'rows': [{'?column?': 'hello'}],
  'fields': [{'name': '?column?', 'dataTypeID': 25}],
  'affectedRows': 0},
 'response_type': 'single'}

In [5]:
# If blocking available (autorespond=True)
# We can pass in multiple statements and receivie multiple responses
# by setting multi=True
pg_headless.query("SELECT 'hello'; SELECT 'goodbye';", multi=True, autorespond=True)

{'status': 'completed',
 'response': [{'rows': [{'?column?': 'hello'}],
   'fields': [{'name': '?column?', 'dataTypeID': 25}],
   'affectedRows': 0},
  {'rows': [{'?column?': 'goodbye'}],
   'fields': [{'name': '?column?', 'dataTypeID': 25}],
   'affectedRows': 0}],
 'response_type': 'multi'}

In [6]:
# If we are in JupyterLite, where there is no blocking
# we need to wait a moment and then view the response
pg_headless.response

{'status': 'completed',
 'response': [{'rows': [{'?column?': 'hello'}],
   'fields': [{'name': '?column?', 'dataTypeID': 25}],
   'affectedRows': 0},
  {'rows': [{'?column?': 'goodbye'}],
   'fields': [{'name': '?column?', 'dataTypeID': 25}],
   'affectedRows': 0}],
 'response_type': 'multi'}

In [7]:
# by default, we assume a single query (multi=False)
pg_headless.query("SELECT 'hello';", autorespond=True)

{'status': 'completed',
 'response': {'rows': [{'?column?': 'hello'}],
  'fields': [{'name': '?column?', 'dataTypeID': 25}],
  'affectedRows': 0},
 'response_type': 'single'}

In [8]:
# If we are in JupyterLite, where there is no blocking
# we need to wait a moment and then view the response
pg_headless.response

{'status': 'completed',
 'response': {'rows': [{'?column?': 'hello'}],
  'fields': [{'name': '?column?', 'dataTypeID': 25}],
  'affectedRows': 0},
 'response_type': 'single'}

In [9]:
%%pglite_magic -w pg_headless
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [10]:
# Show tables
pg_headless.tables(autorespond=True)

['test']

In [11]:
pg_headless.tables(autorespond=False)

'No autoresponse available. View results in response using .table_results()'

In [12]:
pg_headless.table_results()

['test']

In [13]:
# Show table schema
pg_headless.table_schema("test", autorespond=True)

{'status': 'completed',
 'response': {'rows': [{'column_name': 'id',
    'data_type': 'integer',
    'character_maximum_length': None,
    'is_nullable': 'NO',
    'column_default': "nextval('test_id_seq'::regclass)"},
   {'column_name': 'title',
    'data_type': 'character varying',
    'character_maximum_length': None,
    'is_nullable': 'NO',
    'column_default': None}],
  'fields': [{'name': 'column_name', 'dataTypeID': 19},
   {'name': 'data_type', 'dataTypeID': 1043},
   {'name': 'character_maximum_length', 'dataTypeID': 23},
   {'name': 'is_nullable', 'dataTypeID': 1043},
   {'name': 'column_default', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

In [14]:
%%pglite_magic -w pg_headless
INSERT INTO test (title) VALUES ('dummy_headless');


In [15]:
%%pglite_magic -w pg_headless -r
SELECT * FROM test;

{'status': 'completed',
 'response': {'rows': [{'id': 1, 'title': 'dummy_headless'}],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

Use the `.df()` method on the widget to display a query result as a `pandas` dafaframe, if pandas is installed.

In [16]:
pg_headless.df()

,id,title
0,1,dummy_headless


## Inserting Data

If we have a table already defined on the database, and a dataframe that confoms to it, we can add the data in the dataframe to the table as follows:

In [17]:
import pandas as pd

df = pd.DataFrame({"title":["a","b","c"]})

# Insert data from a dataframe into a table that already exists
pg_headless.insert_from_df("test", df, autorespond=True)


{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 3},
 'response_type': 'single'}

In [18]:
# We can insert a dataframe into a pre-existing table
# Broken??
df = pd.DataFrame({"title":["d","e","f"]})
%pglite_df_insert -d df -t test;

In [19]:
pg_headless.query("SELECT * FROM test;", autorespond=True)

{'status': 'completed',
 'response': {'rows': [{'id': 1, 'title': 'dummy_headless'},
   {'id': 2, 'title': 'a'},
   {'id': 3, 'title': 'b'},
   {'id': 4, 'title': 'c'}],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

## Database connections

(Via claude.ai), minimal support for DBAPI2 and SQLAlchmey connections is provided to the extent of allowing read actions using `pd.read_sql()`:

In [20]:
import pandas as pd
from jupyter_anywidget_pglite.dbapi2 import create_connection

conn = create_connection(pg_headless)
pd.read_sql("SELECT * FROM test;", conn)

/var/folders/10/8lp40z8x2hj_cch9j4rk_cpm0000gs/T/ipykernel_96789/3440347236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM test;", conn)


,id,title
0,1,dummy_headless
1,2,a
2,3,b
3,4,c


In [21]:
# Minimal SQLAlchemy connection object support
from jupyter_anywidget_pglite.sqlalchemy_api import create_engine

conn2 = create_engine(pg_headless)
pd.read_sql("SELECT * FROM test;", conn2)

statement is of type: <class 'str'>
Executing query: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'SELECT * FROM test;' AND table_schema = 'public';
        
statement is of type: <class 'str'>
Executing query: SELECT * FROM test;


,id,title
0,1,dummy_headless
1,2,a
2,3,b
3,4,c


In [22]:
with conn2.connect() as connection:
    dialect = connection.dialect  # Get the PGLiteDialect instance
    table_name = "test"  # Change this to the table you want to inspect

    # Call get_columns() method
    cols = dialect.get_columns(connection, table_name)
    table_names = dialect.get_table_names(connection, table_name)
    pk_constraint = dialect.get_pk_constraint(connection, table_name)
    fk = dialect.get_foreign_keys(connection, table_name)
cols, "-------", table_names, "-------", pk_constraint, "------", fk

statement is of type: <class 'str'>
Executing query: 
        SELECT column_name, data_type, is_nullable, column_default 
        FROM information_schema.columns 
        WHERE table_name = 'test'
        AND table_schema = 'public';
        
statement is of type: <class 'str'>
Executing query: SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'test';
statement is of type: <class 'str'>
Executing query: 
        SELECT kcu.column_name
        FROM information_schema.table_constraints AS tc
        JOIN information_schema.key_column_usage AS kcu 
            ON tc.constraint_name = kcu.constraint_name
            AND tc.table_schema = kcu.table_schema
        WHERE tc.constraint_type = 'PRIMARY KEY' 
            AND kcu.table_name = 'test'
            AND kcu.table_schema = 'public';
        
statement is of type: <class 'str'>
Executing query: 
        SELECT 
            kcu.column_name, 
            ccu.table_schema, 
            ccu.table_name, 
            ccu.column_na

([{'name': 'id',
   'type': Integer(),
   'nullable': False,
   'default': "nextval('test_id_seq'::regclass)"},
  {'name': 'title', 'type': String(), 'nullable': False, 'default': None}],
 '-------',
 [],
 '-------',
 {'constrained_columns': ['id'], 'name': 'pk_test'},
 '------',
 [])

In [23]:
# What is this? A write test?
import pandas as pd
pd.DataFrame({"col1":[1,2,3]}).to_sql(name="test2", con=conn2)

statement is of type: <class 'str'>
Executing query: 
        SELECT 1 FROM information_schema.tables 
        WHERE table_name = 'test2' AND table_schema = 'public';
        
statement is of type: <class 'sqlalchemy.sql.elements.quoted_name'>
Inspecting quoted_name instance:
_fallback_getattr: <bound method MemoizedSlots._fallback_getattr of 'test2'>
_memoized_method_lower: <bound method quoted_name._memoized_method_lower of 'test2'>
_memoized_method_upper: <bound method quoted_name._memoized_method_upper of 'test2'>
capitalize: <built-in method capitalize of quoted_name object at 0x1170a18c0>
casefold: <built-in method casefold of quoted_name object at 0x1170a18c0>
center: <built-in method center of quoted_name object at 0x1170a18c0>
construct: <bound method quoted_name.construct of <class 'sqlalchemy.sql.elements.quoted_name'>>
count: <built-in method count of quoted_name object at 0x1170a18c0>
encode: <built-in method encode of quoted_name object at 0x1170a18c0>
endswith: <built-in

KeyError: 'error_message'

## Database persistence

We can persist the database using a browser's `IndexedDB` datastore, passing the name of the indexed database via an `idb=` parameter (Jupyter browser-based UIs; does not work in VSCode).

This parameter can also be used in the inline and panel display database creation steps.

In [ ]:
pg_headless_persist = pglite_headless(idb="pglitetest1")

In [ ]:
# Close / shutdown

## Inline HTML Display (Initialising cell)

We can display the widget inline as a cell output, althoug the utility of this is perhaps a little bit limited?

In [ ]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_inline

pg_inline = pglite_inline()

postgresWidget(response={'status': 'initialising'})

In [ ]:
%%pglite_magic -w pg_inline
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
      );

In [ ]:
%%pglite_magic
INSERT INTO test (title) VALUES ('dummy_inline');

In [ ]:
%pglite_query -D -r -q 'SELECT * FROM test LIMIT 1;'

UsageError: unrecognized arguments: -D


In [ ]:
pg_inline.response

{'rows': [{'id': 1, 'title': 'dummy'}],
 'fields': [{'name': 'id', 'dataTypeID': 23},
  {'name': 'title', 'dataTypeID': 1043}],
 'affectedRows': 0}

## HTML in own panel

In a Juoyter Lab environment, we can use [`jupyterlab-sidecar`](https://github.com/jupyter-widgets/jupyterlab-sidecar) to display the widget HTML UI in its own panel (note: this does not currently work at all in VS Code).

In [ ]:
# Load in the widget panel
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_panel

# Launch it
pg_panel = pglite_panel()
# Wait for it to be ready
pg_panel.ready()

In [ ]:
# About the db
pg_panel.about

{'version': 'PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by emcc (Emscripten gcc/clang-like replacement + linker emulating GNU ld) 3.1.72 (437140d149d9c977ffc8b09dbaf9b0f5a02db190), 32-bit'}

In [ ]:
%%pglite_magic -w pg_panel
CREATE TABLE IF NOT EXISTS test  (
        id serial primary key,
        title varchar not null
)

In [ ]:
# Response from db
pg_panel.response

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [ ]:
%%pglite_magic -r
SELECT * FROM test;

{'status': 'completed',
 'response': {'rows': [],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

In [ ]:
%%pglite_magic
INSERT INTO test (title) VALUES ('dummy_panel');

In [ ]:
%pglite_query -t 15 -r -D -q 'SELECT * FROM test LIMIT 1;'

TimeoutError: Action not completed within the specified timeout.

In [ ]:
%%pglite_magic -m
INSERT INTO test (title) VALUES ('dummy_panel1');
INSERT INTO test (title) VALUES ('dummy_panel2');

In [ ]:
%%pglite_magic -M
INSERT INTO test (title) VALUES ('dummy_panel3');
INSERT INTO test (title) VALUES ('dummy_panel4');

In [ ]:
%%pglite_magic -r -d
SELECT * FROM test;

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4


In [ ]:
pg_panel.df()

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4


In [ ]:
pg_panel.response

{'status': 'completed',
 'response': {'rows': [{'id': 1, 'title': 'dummy_panel1'},
   {'id': 2, 'title': 'dummy_panel2'},
   {'id': 3, 'title': 'dummy_panel3'},
   {'id': 4, 'title': 'dummy_panel4'}],
  'fields': [{'name': 'id', 'dataTypeID': 23},
   {'name': 'title', 'dataTypeID': 1043}],
  'affectedRows': 0},
 'response_type': 'single'}

In [ ]:
df = pg_panel.df()
%pglite_df_insert -d df -t test

In [ ]:
%%pglite_magic -r -d
SELECT * FROM test;

,title
id,
1,dummy_panel1
2,dummy_panel2
3,dummy_panel3
4,dummy_panel4
5,dummy_panel1
6,dummy_panel2
7,dummy_panel3
8,dummy_panel4


In [ ]:
pg_panel.close()